# Northwind Use Case

## SPARQL Training

Northwind is a well-known online e-commerce database which is largely used for training purposes across many database platforms. 

This notebook allows you to execute SPARQL queries against an RDF Graph Database version of Northwind. 

For an introduction to the Northwind database and detailed explanation of the SPARQL queries, please refer to the following article on medium: https://medium.com/gitconnected/northwind-sql-vs-sparql-de662f2e0e7b


## Stories

The following are the stories of the Northwind Use Case.

### As a Human Resources Manager, I want to create a simple report with all employees of the company.

In [ ]:
%%sparql

PREFIX     : <http://www.mysparql.com/resource/northwind/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT
    ?label
    ?lastName
    ?firstName
    ?title
WHERE {
    ?emp a :Employee ;
        rdfs:label ?label ;
        foaf:title ?title ;
        foaf:lastName ?lastName ;
        foaf:firstName ?firstName .
}

### As a Human Resources Manager, I want to know all the employees located in the USA.

In [ ]:
%%sparql 

PREFIX     : <http://www.mysparql.com/resource/northwind/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT
    ?label
    ?lastName
    ?firstName
    ?title
WHERE {
    ?emp a :Employee ;
        rdfs:label ?label ;
        foaf:lastName ?lastName ;
        foaf:firstName ?firstName ;
        foaf:title ?title ;
        :country ?country .
    FILTER(?country = "USA")
}

Note that the same filter can be applied directly as follows:

In [ ]:
%%sparql

PREFIX     : <http://www.mysparql.com/resource/northwind/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT
    ?label
    ?lastName
    ?firstName
    ?title
WHERE {
  ?emp a :Employee ;
      rdfs:label ?label ;
      foaf:lastName ?lastName ;
      foaf:firstName ?firstName ;
      foaf:title ?title ;
      :country "USA" .
}
  

### As a Human Resources Manager, I want to know if the company has employees in the UK.

This query returns a boolean indicating whether a query pattern matches any triples.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>
       
ASK {
    ?emp a :Employee ;
    :country "UK" .
}

### As a Sales Manager, I want to be able to search companies by name.

Note that the query below shows two ways of implementing the filter. You can comment out the first filter and uncomment the second one to verify its result.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?companyName
    ?contactName
    ?address
    ?city
    ?phone
WHERE {
  ?s a :Customer ;
      rdfs:label ?companyLabel ;
      :companyName ?companyName ;
      :contactName ?contactName ;
      :address ?address ;
      :city ?city ;
      :phone ?phone .
    FILTER (REGEX(?companyName, "Rest" , "i" )) # Case Insensitive
    # FILTER CONTAINS (LCASE(?companyName), "rest") # Alternatively, you can use the string function CONTAINS.
}


### As a Sales Manager, I want to create a basic report showing products supplied by companies located in the USA.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productID
    ?productName
    ?unitsInStock
    ?unitPrice
    ?categoryName
    ?contactName
WHERE
{
    ?product a :Product ;
        :productID ?productID ;
        :productName ?productName ;
        :unitsInStock ?unitsInStock ;
        :unitPrice ?unitPrice ;
        :hasCategory ?category ;
        :hasSupplier ?supplier .
    ?category a :Category ;
        :name ?categoryName .
    ?supplier a :Supplier ;
        :contactName ?contactName ;
        :country "USA" .
}

### As a Sales Manager, I want to create a basic report showing customers who placed at least one order.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT DISTINCT
    ?customer
    ?companyName
    ?postalCode
    ?city
    ?country
WHERE {
    ?order a :Order .
    ?customer a :Customer .
    ?order :hasCustomer ?customer .
    ?customer :customerID ?customerID ;
        :companyName ?companyName ;
        :city ?city ;
        :country ?country .
    OPTIONAL {?customer :postalCode ?postalCode} . # Some regions don't use PostalCode.
}
ORDER BY
    ?customer


### As a Marketing Manager, I want to create a basic report showing customers who never placed an order.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT DISTINCT
    ?customer
    ?companyName
    ?postalCode
    ?city
    ?country
WHERE {
    ?customer a :Customer .
    ?customer :customerID ?customerID ;
        :companyName ?companyName ;
        :city ?city ;
        :country ?country .
        OPTIONAL {?customer :postalCode ?postalCode} . # Some regions don't use PostalCode.
        OPTIONAL {
    ?order a  :Order .
    ?customer ^:hasCustomer ?order # for customers with no orders, ?order variable will be empty (not bound).
}
    FILTER (!BOUND(?order)) # Checks if variable is not bound to a value.
}
ORDER BY
    ?customer



The same result can be obtained by using MINUS.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?customer
    ?companyName
    ?postalCode
    ?city
    ?country
WHERE {
    {
        ?customer a :Customer ; # All customers
            :customerID ?customerID ;
            :companyName ?companyName ;
            :city ?city ;
            :country ?country .
    } MINUS {
        ?customer a :Customer . # Customers who placed orders
        ?order a  :Order .
        ?order :hasCustomer ?customer .
    }
}

### As a Marketing Manager, I want to search products by name or a combination of identification number and price.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productName
    (STR(?unitPrice) AS ?strUnitPrice) # converting integer to string
    ?supplierName
    ?region
    ?country
WHERE {
    ?s a :Product ;
        :productName ?productName ;
        :productID ?productID ;
        :hasSupplier ?supplier ; # Joining on supplier
        :unitPrice ?unitPrice .
    # getting supplier properties
    ?supplier :companyName ?supplierName ;
        :country ?country ;
    OPTIONAL {?supplier  :region ?region }. # not all suppliers have region
    FILTER((REGEX(?productName, "^T", "i")) || (?productID = 46 && ?unitPrice > 16)) . # Logical operators
}


### As a Marketing Manager, I want to know which products are in a given price range.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productName
    ?companyName
    ?unitPrice
WHERE {
    ?s a :Product ;
        :productID ?productID ;
        :productName ?productName ;
        :hasSupplier ?supplier ;
        :unitPrice ?unitPrice .
    ?supplier a :Supplier ;
        :companyName ?companyName ;
        :supplierID ?supplierID .
    FILTER (?unitPrice >= 18 && ?unitPrice <= 20)
}

### As a Marketing Manager, I want to create a list of all suppliers located in Japan or Italy.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?companyName
    ?country
WHERE {
    ?s a :Supplier ;
        :companyName ?companyName ;
        :country ?country .
    FILTER (UCASE(?country) = "JAPAN" || ?country = "Italy") # case sensitive
}



### As a Marketing Manager, I want to produce a report containing all suppliers.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?companyName
    ?fax
WHERE {
    ?s a :Supplier ;
        :companyName ?companyName ;
    OPTIONAL {?s :fax ?fax} .
}

### As a Marketing Manager, I want to produce a report containing all suppliers that have a fax number.
Note: Fax was a machine from the 90s able to scan and transmit a document over the phone line.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?companyName
    ?fax
WHERE {
    ?s a :Supplier ;
        :companyName ?companyName ;
        :fax ?fax .
}

### As a Marketing Manager, I want to produce a report containing all suppliers that don't have a fax number.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?companyName
    ?fax
WHERE {
    ?s a :Supplier ;
        :companyName ?companyName ;
    OPTIONAL {?s :fax ?fax} .
    FILTER (!BOUND(?fax))
}

The same result can be obtained by using the NOT EXISTS filter below.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?companyName
    ?fax
WHERE {
    ?s a :Supplier ;
        :companyName ?companyName .
    FILTER NOT EXISTS {
        SELECT
            ?companyName
        WHERE {
            ?s a :Supplier ;
            :companyName ?companyName ;
            :fax ?fax .
        }
    }
}

### As a Marketing Manager, I want to produce a report of products grouped by category and sorted by unit price descending.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productName
    ?categoryName
    ?unitPrice
WHERE {
    ?s a :Product ;
        :productID ?productID ;
        :productName ?productName ;
        :unitPrice ?unitPrice ;
        :hasCategory ?category .
    ?category :name ?categoryName .
}
ORDER BY
    ASC(?categoryName)
    DESC(?unitPrice)

### As a Marketing Manager, I want to produce a report with all countries I buy from.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT DISTINCT
    ?country
WHERE{
    ?s a :Supplier ;
        :country ?country .
}
ORDER BY
    ?country # Default sorting

### As a Data Stward, I want to generate an identification code for each of our employees.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT
    (CONCAT (?firstName, " ", ?lastName) AS ?fullName)
    ?code
WHERE {
    ?s a :Employee ;
        foaf:firstName ?firstName ;
        foaf:lastName ?lastName ;
        rdfs:label ?employeeLabel ;
        :extension ?extension ;
        :country ?country ;
        OPTIONAL {?s :region ?region } .
    BIND(CONCAT(SUBSTR(?firstName,1,1), SUBSTR(?lastName,1,3), "-", ?extension, "-", IF(!BOUND(?region),
    CONCAT("INT-", ?country), ?region)) AS ?code)
}
ORDER BY
    ?lastName

### As a Sales Manager, I want to create a report with the top 5 largest quantity of a product sold in a single order.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productName
    ?orderID
    ?orderDate
    ?quantity
    ?unitsInStock
WHERE {
    ?orderDetail a :OrderDetail .
    ?order a :Order .
    ?product a :Product .
    ?orderDetail :quantity ?quantity ;
        :belongsToOrder ?order ;
        :hasProduct ?product .
    ?order :orderID ?orderID ;
        :orderDate ?orderDate .
    ?product :unitsInStock ?unitsInStock ;
        :productName ?productName .
}
ORDER BY
    DESC(?quantity)
    DESC(?orderDate)
LIMIT 5

### As a Sales Manager, I want to retrieve the second page of a report with the top largest quantity of a product sold in a single order.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productName
    ?orderID
    ?orderDate
    ?quantity
    ?unitsInStock
WHERE {
    ?orderDetail a :OrderDetail .
    ?order a :Order .
    ?product a :Product .
    ?orderDetail :quantity ?quantity ;
        :belongsToOrder ?order ;
        :hasProduct ?product .
    ?order :orderID ?orderID ;
        :orderDate ?orderDate .
    ?product :unitsInStock ?unitsInStock ;
        :productName ?productName .
}
ORDER BY
    DESC(?quantity)
    DESC(?orderDate)
OFFSET 5
LIMIT 5

### As a Sales Manager, I want to know the total number of suppliers.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT (COUNT(1) AS ?supplierCount)
WHERE{
    ?s a :Supplier .
}

### As a Sales Manager, I want to know the number of countries I buy from.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT (COUNT(DISTINCT ?country) AS ?countryCount)
WHERE{
    ?s a :Supplier ;
        :country ?country .
}

### As a Sales Manager, I want to know the top 5 most sold products.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productID
    (SUM(?quantity) AS ?totalQtySold)
WHERE {
    ?order a :OrderDetail ;
        :quantity ?quantity ;
        :hasProduct ?product .
    ?product :productID ?productID .
}
GROUP BY
    ?productID
ORDER BY
    DESC(?totalQtySold)
LIMIT 5

### As a Sales Manager, I want to know the top 5 largest orders shipped to the USA.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?orderID
    (ROUND(SUM(?unitPrice * ?quantity * (1 - ?discount))) AS ?total)
WHERE {
    ?order a :Order ;
        :orderID ?orderID ;
        :shipCountry "USA" .
    ?orderDetail a :OrderDetail ;
        :belongsToOrder ?order ;
        :unitPrice ?unitPrice ;
        :quantity ?quantity ;
        :discount ?discount .
}
GROUP BY
    ?orderID
ORDER BY
    DESC(?total)
LIMIT 5


### As a Sales Manager, I want to know the orders over 10K shipped to the USA.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
?orderID
(ROUND(SUM(?unitPrice * ?quantity * (1 - ?discount))) AS ?total)
WHERE {
    ?order a :Order ;
        :orderID ?orderID ;
        :shipCountry "USA" .
    ?orderDetail a :OrderDetail ;
        :belongsToOrder ?order ;
        :unitPrice ?unitPrice ;
        :quantity ?quantity ;
        :discount ?discount .
}
GROUP BY
    ?orderID
HAVING (SUM(?unitPrice * ?quantity * (1 - ?discount)) > 10000)
ORDER BY
    DESC(?total)
    

### As a Sales Manager, I want to know the top 5 supplier representative by number of products sold.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?supplierContactName
    (COUNT(?product) as ?productCount)
WHERE
{
    ?product a  :Product ;
        :hasSupplier ?supplier .
    ?supplier a :Supplier ;
        :contactName ?supplierContactName .
}
GROUP BY
    ?supplierContactName
ORDER BY
    DESC(?productCount)
LIMIT 5

## Recommendation

The next three queries focus on recommendations.




### As a Sales Manager, I want to know which products were bought together in the same order.
Query: Customers who bought product-61 also bought which products in the same order and how many times?

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productA
    ?productB
    (COUNT (*) AS ?productBCount)
WHERE {
    ?productA ^:hasProduct/:belongsToOrder/^(^:hasProduct/:belongsToOrder) ?productB ;
        :productID ?productID .
    FILTER (?productA != ?productB && ?productA = :product-61) # Filtering on product-61 for testing
}
GROUP BY
    ?productA
    ?productB
ORDER BY
    DESC(?productBCount) ?productA ?productB # Most frequent at the top

### As a Sales Manager, I want to know which products were bought together across all orders.

Query: Customers who bought product-61 also bought which products across all orders and how many times?

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productA
    ?productB
    (COUNT (*) AS ?productBCount)
WHERE {
    ?productA ^:hasProduct/:belongsToOrder/:hasCustomer/^(^:hasProduct/:belongsToOrder/:hasCustomer) ?productB ;
        :productID ?productID .
    FILTER (?productA != ?productB && ?productA = :product-61) # Filtering on product-61 for testing purposes
}
GROUP BY
    ?productA
    ?productB
ORDER BY
    DESC(?productBCount) ?productA ?productB

### As a Sales Manager, I want to know how many times two given products where bought by the same customer. 

Query: How many times products 2 and 61 where bought by the same customer.

*Refer to the following article for more details (6 steps) on how the query below was designed: 
https://medium.com/gitconnected/northwind-sql-vs-sparql-de662f2e0e7b*

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT (COUNT (1) AS ?Count)
WHERE { 
    :product-2 ^:hasProduct/:belongsToOrder/:hasCustomer/^(^:hasProduct/:belongsToOrder/:hasCustomer) :product-61 
}


### As a Sales Manager, I want to know the contact details of suppliers, customers and employees to send out Xmas cards.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT
    ?contactName
    ?address
    ?city
    ?postalCode
    ?country
WHERE {
    {
        ?supplier a :Supplier ;
        :contactName ?contactName ;
        :address ?address ;
        :city ?city ;
        :postalCode ?postalCode ;
        :country ?country .
    } UNION {
        ?customer a :Customer ;
        :contactName ?contactName ;
        :address ?address ;
        :city ?city ;
        :postalCode ?postalCode ;
        :country ?country .
    } UNION {
        ?employee a :Employee ;
        foaf:firstName ?firstName ;
        foaf:lastName ?lastName ;
        :address ?address ;
        :city ?city ;
        :postalCode ?postalCode ;
        :country ?country .
        BIND (CONCAT (?firstName, " ", ?lastName) AS ?contactName)
    }
}

### As a Sales Manager, I want to know all products that belong to the Seafood category an their quantity in stock.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    ?productName
    ?unitPrice
    ?unitsInStock
WHERE { # outer query
    ?product a :Product ;
        :productName ?productName ;
        :unitPrice ?unitPrice ;
        :unitsInStock ?unitsInStock ;
        :hasCategory ?category .
    { # inner query
        SELECT
            ?category
        WHERE {
            ?category a :Category ;
                :categoryID ?categoryID ;
                :name "Seafood" .
        }
    }
}
ORDER BY
    ?productName

### As a Sales Manager, I want to calculate the average number of orders processed per year.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

SELECT
    (AVG(?orderCount) AS ?avgCount)
    (MIN(?orderYear) AS ?startYear)
    (MAX(?orderYear) AS ?endYear)
{
    SELECT ?orderYear (count(?order) AS ?orderCount)
    WHERE {
        ?order a :Order ;
            :orderDate ?orderDate ;
        BIND(year(?orderDate) AS ?orderYear)
    }
    GROUP BY
        ?orderYear
}

### As a Sales Representative, I want to be able to insert a new customer.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

INSERT DATA {
    :customer-AAAAA a :Customer ;
    rdfs:label "customer-AAAAA" ;
    :customerID "AAAAA" ;
    :companyName "Northwind" ;
    :contactName "John Lennon" ;
    :contactTitle "CTO" ;
    :address "Abbey Road" ;
    :city "London" .
}

Checking if new customer has been added successfully

In [ ]:
%%sparql

PREFIX   : <http://www.mysparql.com/resource/northwind/>
DESCRIBE :customer-AAAAA

### As a Sales Representative, I want to be able to update an existing customer.

In [ ]:
%%sparql

# Step 1: Insert new triple for for the properties not included the original insert query

PREFIX : <http://www.mysparql.com/resource/northwind/>

INSERT DATA {
    :customer-AAAAA a :Customer ;
    :country "UK" ;
    :postalCode "SW1A 2AA" .
}

In [ ]:
%%sparql

# Step 2: Update the property values added.

PREFIX : <http://www.mysparql.com/resource/northwind/>

DELETE {
    :customer-AAAAA :address ?oldAddress
}
INSERT {
    :customer-AAAAA :address ?newAddress
}
WHERE {
    :customer-AAAAA a :Customer ;
    :address ?oldAddress ;
    BIND("10 Downing Road" AS ?newAddress) .
}

Checking if existing customer has been updated successfully

In [ ]:
%%sparql

PREFIX   : <http://www.mysparql.com/resource/northwind/>
DESCRIBE :customer-AAAAA

### As a Sales Representative, I want to be able to delete an existing customer.

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>

DELETE {
    :customer-AAAAA ?p ?s 
}
WHERE {
    :customer-AAAAA ?p ?s .
}

Checking if existing customer has been deleted successfully

In [ ]:
%%sparql

PREFIX : <http://www.mysparql.com/resource/northwind/>
DESCRIBE :customer-AAAAA

### ----- END -----

#### Script used to load the data
%load was failing for turtle and ntriple imports, so I loaded the data manually in here. 

In [ ]:
%%sparql

# PREFIX : <http://www.mysparql.com/resource/northwind/>
# # DROP GRAPH :NorthwindGraph
# INSERT DATA {
#     GRAPH :NorthwindGraph {
#     # 3 Batches of 10K triples pasted here. 
#     }
# }